In [1]:
import sys
import os
sys.path.append(os.path.abspath('..')) # add parent folder to path
import backtrader as bt
import pandas as pd
import matplotlib
matplotlib.use('QT5Agg')


In [2]:
dataname='../data/GC=F_com_disagg.csv'
# data = OHLCAVWithDisaggCoT(dataname=dataname)

cerebro = bt.Cerebro()
df = pd.read_csv(dataname, index_col=0)
df.index = pd.to_datetime(df.index)
# df


In [3]:
from data_feeds.CoT_Dissag import PandasData_more
from strategies.btStrategy import MyStrategy

data = PandasData_more(dataname=df)

# data = OHLCAVWithDisaggCoT(dataname = dataname)

cerebro.addstrategy(MyStrategy)

cerebro.adddata(data)

# cerebro.addstrategy(MyStrategy)
# cerebro.add_signal(bt.SIGNAL_LONG, SMACloseSignal, period = 10)
# cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal)

# 初始资金 100,000,000
cerebro.broker.setcash(1e6)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

cerebro.addobserver(bt.observers.BuySell) # 买卖交易点
cerebro.addobserver(bt.observers.Value) # 价值


cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤


In [4]:
# 启动回测
result = cerebro.run()
cerebro.plot(iplot= False)
# 从返回的 result 中提取回测结果
strat = result[0]
# 返回日度收益率序列
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())



init
--------- 打印 self.datas 第一个数据表格的 lines ----------
('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime', 'Tot_Rept_Positions_Long_All', 'Tot_Rept_Positions_Short_All', 'M_Money_Positions_Long_All', 'M_Money_Positions_Short_All', 'Pct_of_OI_M_Money_Short_All', 'Pct_of_OI_M_Money_Long_All', 'Pct_of_OI_Tot_Rept_Short_All', 'Pct_of_OI_Tot_Rept_Long_All')
--------------- AnnualReturn -----------------
OrderedDict([(2019, -0.0009973193870866925), (2020, -0.009919295595442468), (2021, 0.0017986881518472497), (2022, 0.0011767323686870945)])
--------------- SharpeRatio -----------------
OrderedDict([('sharperatio', -2.5517765550806404)])
--------------- DrawDown -----------------
AutoOrderedDict([('len', 67), ('drawdown', 1.4086501277883314), ('moneydown', 14174.011571275303), ('max', AutoOrderedDict([('len', 586), ('drawdown', 2.3611482812911273), ('moneydown', 23622.352193525876)]))])
